In [7]:
import sys
import numpy as np
import scipy.io
from pyDOE import lhs
from torch import Tensor, ones, stack, load
from torch.autograd import grad
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import h5py as h5
import os

In [8]:
sys.path.append('/Users/juanesteban') 
import PINNFramework as pf

In [18]:
lb= np.array(list(zip(np.linspace(-1,-1, y_dim), np.linspace(-1,1, y_dim))))
rb= np.array(list(zip(np.linspace(1,1, y_dim), np.linspace(-1,1, y_dim))))
ub = np.array(list(zip(np.linspace(-1,1, y_dim), np.linspace(1,1, y_dim))))
lwb =  np.array(list(zip(np.linspace(-1,1, y_dim), np.linspace(-1,-1, y_dim))))
Bdy_coor = np.concatenate((lb,rb,ub,lwb))

In [34]:
x_dim = 4
y_dim = 4
nsteps = 20
x_0 = np.linspace(-1,1, y_dim)
y_0 = np.linspace(-1,1, y_dim)
dt = 0.001
X, Y = np.meshgrid(x_0, y_0)
X_0 = X.reshape(-1)
Y_0 = Y.reshape(-1)
idx_x = np.random.choice(len(X_0), 3, replace=False)
x = np.array([X_0[idx_x]]).T
y = np.array([Y_0[idx_x]]).T
print(x.shape)
sol = np.array(Psi(X_0,Y_0,0,1))
np.concatenate([x,y,x],axis=1).shape

(3, 1)


(3, 3)

In [12]:
#Create 2-D Dataset from the analytical solution 
Bdy_x = [-1,1]
Bdy_y = [-1,1]
inv_fthsqrt_pi = 1 / np.pi ** (1. / 4.)
inv_sqrt_two = 1. / np.sqrt(2.)

def phi0(x,f):
    root_f = f ** (1./4.)
    return root_f*inv_fthsqrt_pi * np.exp(- x[0] * f * x[0] / 2.0)

def phi1(x,f):
    return  phi0(x,f) * inv_sqrt_two * (np.sqrt(f) * 2.0 * x)

def Phi0(x, y, f):
    return phi0(x,f) * phi0(y,f)

def Phi1(x, y, f):
    return phi1(x,f) * phi1(y,f)


def Psi(x, y, t,f):
    return inv_sqrt_two * (np.exp(-1j*f*t) * Phi0(x, y,f) + np.exp(-1j*3*f*t)*Phi1(x, y,f))
x_dim = 2
y_dim = 2
nsteps = 20
x = [-1]
y = np.linspace(-3,3, y_dim)
dt = 0.001
X, Y = np.meshgrid(x, y)
X = X.reshape(-1)
Y = Y.reshape(-1)
for t in range(0,nsteps):
    if t % 200 == 0:
        print("t", t)
sol = Psi(X, Y, t, 1)
list(zip(X,Y,T))

t 0


NameError: name 'T' is not defined

In [87]:
x_bdy=[-1,1]
y_bdy=[-1,1]
x_dim = 10
y_dim = 10
nsteps= 10
ft=0.01
xl= np.array(list(zip(np.linspace(x_bdy[0],x_bdy[0], x_dim),
                           np.linspace(y_bdy[0],y_bdy[1], y_dim))))
xr= np.array(list(zip(np.linspace(x_bdy[1],x_bdy[1], x_dim),
                           np.linspace(y_bdy[0],y_bdy[1], y_dim))))
yu = np.array(list(zip(np.linspace(x_bdy[0],x_bdy[1], x_dim),
                           np.linspace(y_bdy[1],y_bdy[1], y_dim))))
ylw = np.array(list(zip(np.linspace(x_bdy[0],x_bdy[1], x_dim),
                           np.linspace(y_bdy[0],y_bdy[0], y_dim))))
x_b = np.array(np.concatenate([xl,xr,yu,ylw],axis=0))
T = np.linspace(0,nsteps*dt,nsteps)
xyt = np.array([[[x_b[i][0],x_b[i][0],T[k]]for i in range(len(x_b))]for k in range (len(T))])
xyt= xyt.reshape(xyt.shape[0]*xyt.shape[1], xyt.shape[2])
index_bdy = np.random.choice(len(xyt), 5, replace=False)
Bdy_training = xyt[index_bdy]

In [101]:
nb=10
np.zeros((10,2))

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [13]:
data = scipy.io.loadmat('/Users/juanesteban/PINNS/NeuralSolvers/examples/1D_Schroedinger/NLS.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Fri Nov 10 20:10:33 2017',
 '__version__': '1.0',
 '__globals__': [],
 'tt': array([[0.        , 0.00785398, 0.01570796, 0.02356194, 0.03141593,
         0.03926991, 0.04712389, 0.05497787, 0.06283185, 0.07068583,
         0.07853982, 0.0863938 , 0.09424778, 0.10210176, 0.10995574,
         0.11780972, 0.12566371, 0.13351769, 0.14137167, 0.14922565,
         0.15707963, 0.16493361, 0.1727876 , 0.18064158, 0.18849556,
         0.19634954, 0.20420352, 0.2120575 , 0.21991149, 0.22776547,
         0.23561945, 0.24347343, 0.25132741, 0.25918139, 0.26703538,
         0.27488936, 0.28274334, 0.29059732, 0.2984513 , 0.30630528,
         0.31415927, 0.32201325, 0.32986723, 0.33772121, 0.34557519,
         0.35342917, 0.36128316, 0.36913714, 0.37699112, 0.3848451 ,
         0.39269908, 0.40055306, 0.40840704, 0.41626103, 0.42411501,
         0.43196899, 0.43982297, 0.44767695, 0.45553093, 0.46338492,
         0.4712389 , 0.4790

In [14]:
class BoundaryConditionDataset(Dataset):

    def __init__(self, nb, x_bdy, y_bdy, x_dim, y_dim):
        """
        Constructor of the Boundary condition dataset, with x_bdy an array with the 
        lower and uper bound in the x direction and respectively y_bdy. Only for square domain.
        """
        xl= np.array(list(zip(np.linspace(x_bdy[0],x_bdy[0], x_dim), 
                                   np.linspace(y_bdy[0],y_bdy[1], y_dim))))
        xr= np.array(list(zip(np.linspace(x_bdy[1],x_bdy[1], x_dim), 
                                   np.linspace(y_bdy[0],y_bdy[1], y_dim))))
        yu = np.array(list(zip(np.linspace(x_bdy[0],x_bdy[1], x_dim), 
                                   np.linspace(y_bdy[1],y_bdy[1], y_dim))))
        ylw =  np.array(list(zip(np.linspace(x_bdy[0],x_bdy[1], x_dim), 
                                   np.linspace(y_bdy[0],y_bdy[0], y_dim))))
        self.Bdy_coor = np.concatenate((xl,xr,yu,ylw))
        self.Bdy_training =  np.random.choice(len(self.Bdy_coor), nb, replace=False)
    def __getitem__(self, idx):
        """
        Returns data for initial state
        """
        return Tensor(self.Bdy_training).float()

    def __len__(self):
        """
        There exists no batch processing. So the size is 1
        """
        return 1


In [15]:
class InitialConditionDataset(Dataset):

    def __init__(self, nx ,x_bdy, y_bdy, x_dim, y_dim):
        """
        Constructor of the boundary condition dataset
        Args:
          n0 (int)
        """
        super(type(self)).__init__()
        #data = scipy.io.loadmat('/Users/juanesteban/PINNS/NeuralSolvers/examples/1D_Schroedinger/NLS.mat')
        #x = data['x'].flatten()[:, None]
        #t = data['tt'].flatten()[:, None]
        x_0 = np.linspace(-1,1, x_dim)
        y_0 = np.linspace(-1,1, y_dim)
        dt = 0.001
        X, Y = np.meshgrid(x_0, y_0)
        X_0 = X.reshape(-1)
        Y_0 = Y.reshape(-1)
        idx_x = np.random.choice(len(X_0), nx, replace=False)
        self.x = np.array(X_0[idx_x])
        self.y = np.array(Y_0[idx_x])
        sol = Psi(self.x,self.y,0,1) #Psi(x,y,t=0,f)
        self.u = sol.real
        self.v = sol.imag
        self.t = np.zeros(len(self.x))
        print(self.t)

    def __len__(self):
        """
        There exists no batch processing. So the size is 1
        """
        return 1

    def __getitem__(self,idx):
        x = list(zip([self.x, self.y,self.t]))
        y = list(zip([self.u, self.v]))
        return Tensor(x).float(), Tensor(y)



In [16]:
class PDEDataset(Dataset):
    def __init__(self, nf, lb, ub):
        self.xf = lb + (ub - lb) * lhs(2, nf)

    def __getitem__(self, idx):
        """
        Returns data for initial state
        """
        return Tensor(self.xf).float()

    def __len__(self):
        """
        There exists no batch processing. So the size is 1
        """
        return 1


if __name__ == "__main__":
    # Domain bounds
    lb = np.array([-5.0, 0.0])
    ub = np.array([5.0, np.pi / 2])
    # initial condition
    ic_dataset = InitialConditionDataset(nx = 50, x_bdy=np.array([-1,1]),
                                         y_bdy=np.array([-1,1]), x_dim =200, y_dim=200)                                         
    initial_condition = pf.InitialCondition(ic_dataset)
    # boundary conditions
    bc_dataset = BoundaryConditionDataset(nb=50, x_bdy=np.array([-1,1]),y_bdy=np.array([-1,1])
                                          ,x_dim=200,y_dim=200)
    periodic_bc_u = pf.PeriodicBC(bc_dataset, 0, "u periodic boundary condition")
    periodic_bc_v = pf.PeriodicBC(bc_dataset, 1, "v periodic boundary condition")
    periodic_bc_u_x = pf.PeriodicBC(bc_dataset, 0, "u_x periodic boundary condition", 1, 0)
    periodic_bc_v_x = pf.PeriodicBC(bc_dataset, 1, "v_x periodic boundary condition", 1, 0)
    # PDE
    pde_dataset = PDEDataset(200, lb, ub)


    def schroedinger1d(x, u):
        pred = u
        u = pred[:, 0]
        v = pred[:, 1]
        print("x:", x.shape)
        print("u:", v.shape)
        grads = ones(u.shape, device=pred.device) # move to the same device as prediction
        grad_u = grad(u, x, create_graph=True, grad_outputs=grads)[0]
        grad_v = grad(v, x, create_graph=True, grad_outputs=grads)[0]

        # calculate first order derivatives
        u_x = grad_u[:, 0]
        print("u_x", u_x.shape)
        u_t = grad_u[:, 1]
        v_x = grad_v[:, 0]
        v_t = grad_v[:, 1]

        # calculate second order derivatives
        grad_u_x = grad(u_x, x, create_graph=True, grad_outputs=grads)[0]
        grad_v_x = grad(v_x, x, create_graph=True, grad_outputs=grads)[0]

        u_xx = grad_u_x[:, 0]
        print("u_xx", u_xx.shape)
        v_xx = grad_v_x[:, 0]
        f_u = u_t + u_x#0.5 * v_xx + (u ** 2 + v ** 2) * v
        print("f_u.shape", f_u.shape)
        f_v = v_t - 0.5 * u_xx - (u ** 2 + v ** 2) * u
        print(f_u)
        return stack([f_u, f_v], 1)  # concatenate real part and imaginary part


    pde_loss = pf.PDELoss(pde_dataset, schroedinger1d)
    model = pf.models.MLP(input_size=3, output_size=2, hidden_size=100, num_hidden=4, lb=lb, ub=ub)
    pinn = pf.PINN(model, 2, 2, pde_loss, initial_condition, [periodic_bc_u,
                                                              periodic_bc_v,
                                                              periodic_bc_u_x,
                                                              periodic_bc_v_x], use_gpu=False)
    pinn.fit(2, 'Adam', 1e-3)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


RuntimeError: The size of tensor a (50) must match the size of tensor b (2) at non-singleton dimension 2